In [4]:
import numpy as np
import pandas as pd
import json

fine_path = 'data/train_fine.txt'
dic = set()
count = 1
l = 0
with open(fine_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        l = max(0, len(data['title']))
        for key in data['key_attr']:
            if key not in dic:
                dic.add(key)
                count += 1

print(key)
print(dic)


领型
{'裙长', '袖长', '裤长', '裤型', '衣长', '闭合方式', '版型', '鞋帮高度', '领型', '裤门襟', '穿着方式', '类别'}


In [5]:
print(l)

19


In [9]:
import collections

key_dic = collections.defaultdict(int)
i = 0
with open('data/attr_to_attrvals.json', 'r', encoding='utf-8') as f:
    key_attr = json.load(f)
    for key in key_attr:
        key_dic[key] = i
        i += 1
print(key_dic)


defaultdict(<class 'int'>, {'领型': 0, '袖长': 1, '衣长': 2, '版型': 3, '裙长': 4, '穿着方式': 5, '类别': 6, '裤型': 7, '裤长': 8, '裤门襟': 9, '闭合方式': 10, '鞋帮高度': 11})


In [10]:
import os 
import numpy as np
import json

data_path_1 = './data/train_fine.txt'
data_path_2 = './data/train_coarse.txt'

data_pos_list = []
data_neg_list = []

with open(data_path_1, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        if data['match']['图文'] == 1:
            data_pos_list.append(data)
        else:
            data_neg_list.append(data)

with open(data_path_2, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        if data['match']['图文'] == 1:
            data_pos_list.append(data)
        else:
            data_neg_list.append(data)

np.random.shuffle(data_pos_list)
x_train_list = data_pos_list[:len(data_pos_list)-len(data_neg_list)]
x_val_list = data_pos_list[len(data_pos_list)-len(data_neg_list):] + data_neg_list
np.random.shuffle(x_val_list)

pre_ret = []
train_ret = []
val_ret = []
for dic in x_train_list:
    pre_ret.append(json.dumps(dic, ensure_ascii=False)+'\n')

l = len(x_val_list)-4000
for i in range(l):
    dic = x_val_list[i]
    train_ret.append(json.dumps(dic, ensure_ascii=False)+'\n')

for i in range(l, len(x_val_list)):
    dic = x_val_list[i]
    val_ret.append(json.dumps(dic, ensure_ascii=False)+'\n')

with open('./data/pretrain_match.txt', 'w', encoding='utf-8') as f:
    f.writelines(pre_ret)

with open('./data/train_match.txt', 'w', encoding='utf-8') as f:
    f.writelines(train_ret)

with open('./data/val_match.txt', 'w', encoding='utf-8') as f:
    f.writelines(val_ret)

print(len(pre_ret))
print(len(train_ret))
print(len(val_ret))
print(len(pre_ret) + len(train_ret) + len(val_ret))

129176
16824
4000
150000
